here i attempt to write a REST2 factory for a canonical openmm.system (i.e. containing `HarmonicBondForce`, `HarmonicAngleForce`, `PeriodicTorsionForce`, `NonbondedForce`)

In [1]:
from openmmtools.testsystems import AlanineDipeptideExplicit

define a vacuum alanine dipeptide

In [2]:
ala = AlanineDipeptideExplicit()
sys = ala.system

In [3]:
from perses.annihilation.rest import RESTTopologyFactory

query the topology

In [4]:
# ala.topology
# for atom in ala.topology.atoms():
#     print(atom)

I'll make the ALA residue 'solute' for REST2

remove the `CMMForce`

In [5]:
ala.system.removeForce(4)

make the factory

In [6]:
factory = RESTTopologyFactory(ala.system, solute_region=list(range(6, 16)), use_dispersion_correction=True)

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]
INFO:REST:No unknown forces.


pull the system

In [7]:
REST_system = factory.REST_system

query the forces to make sure it _looks_ right

In [8]:
REST_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x7fe5fc2ab750> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x7fe5fc2ab6f0> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x7fe5fc2ab840> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fe5fc2ab6c0> >]

In [9]:
og_nbf = ala.system.getForce(3)
REST_nbf = REST_system.getForce(3)

In [10]:
og_nbf.getLJPMEParameters()

[0.0, 0, 0, 0]

In [11]:
REST_nbf.getLJPMEParameters()

[0.0, 0, 0, 0]

In [12]:
og_nbf.getNonbondedMethod()

4

In [13]:
REST_nbf.getNonbondedMethod()

4

In [14]:
og_nbf.getPMEParameters()

[Quantity(value=0.0, unit=/nanometer), 0, 0, 0]

In [15]:
REST_nbf.getPMEParameters()

[Quantity(value=0.0, unit=/nanometer), 0, 0, 0]

In [16]:
og_nbf.getReactionFieldDielectric()

78.3

In [17]:
REST_nbf.getReactionFieldDielectric()

78.3

In [18]:
og_nbf.getSwitchingDistance(
)

Quantity(value=0.8500000000000001, unit=nanometer)

In [19]:
REST_nbf.getSwitchingDistance()

Quantity(value=0.8500000000000001, unit=nanometer)

In [20]:
og_nbf.getUseSwitchingFunction()

True

In [21]:
REST_nbf.getUseDispersionCorrection()

True

In [33]:
og_nbf.getUseDispersionCorrection()

True

In [22]:
og_nbf.getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [23]:
REST_nbf.getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [24]:
og_nbf.getUseSwitchingFunction()

True

In [25]:
REST_nbf.getUseSwitchingFunction()

True

all of the nonbonded forces must have the same number of particles

In [26]:
num_particles = REST_system.getNumParticles()
#assert num_particles == factory._out_system_forces['CustomNonbondedForce'].getNumParticles()
assert num_particles == factory._out_system_forces['NonbondedForce'].getNumParticles()

## TODO:

here is a few things that have to be done...

- Given the default Global variables, do energy component bookkeeping with the REST system and the original system. Specifically, the first `CustomBondForce` must match the `HarmonicBondForce` (same with the second and third forces with the `HarmonicAngle` and `PeriodicTorsionForce`s). The sum of the `NonbondedForce`, the `CustomNonbondedForce`, and the `CustomBondForce` (last one) must be equal to the energy of the original system's `NonbondedForce`.

- Create a subclass of the `AlchemicalState` to implement the appropriate parameters for `solute_scale` and `inter_scale` (which are `GlobalParameters` of the `REST_system`) and perform Repex.

- Run REST2!

## Run energy component bookkeeping

In [29]:
from openmmtools.states import CompoundThermodynamicState, SamplerState, ThermodynamicState
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from openmmtools.integrators import LangevinIntegrator
from simtk import unit
from openmmtools import cache
temp = 300 * unit.kelvin
from perses.tests.utils import compute_potential_components
import copy
from openmmtools.constants import kB
beta = 1/(kB * temp)
from perses.dispersed.feptasks import minimize
from coddiwomple.openmm.states import OpenMMPDFState
from perses.dispersed.utils import configure_platform
from coddiwomple.openmm.utils import get_dummy_integrator
from openmmtools import utils

In [30]:
from perses.tests.test_topology_proposal import generate_atp

Potential components for the vanilla ala system

In [31]:
thermostate = ThermodynamicState(system=ala.system, temperature=temp)
sampler_state = SamplerState(ala.positions, box_vectors = ala.system.getDefaultPeriodicBoxVectors())
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
minimize(thermostate, sampler_state)
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 4.620441504477346), ('HarmonicAngleForce', 1.6731238767690715), ('PeriodicTorsionForce', 2.0709313392797806), ('NonbondedForce', -13196.124956744168), ('AndersenThermostat', 0.0)]


Potential components for the REST system

In [32]:
thermostate = ThermodynamicState(system=REST_system, temperature=temp)
# Don't re-initialize the sampler state 
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
# Don't minimize again
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('CustomBondForce', 4.620441504477346), ('CustomAngleForce', 1.673123876769072), ('CustomTorsionForce', 2.07093133927978), ('NonbondedForce', -13196.124956744174), ('AndersenThermostat', 0.0)]


In [16]:
# REST Nonbonded + CustomNonbonded + CustomBond = -50.949

## Run REST2 on capped THR->ALA (flattened torsions/exceptions) in vacuum

In [17]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import copy

In [18]:
# Load in r-htf at endstate = 0
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/11/0"
htf = pickle.load(open(os.path.join(out_dir, f"0_vacuum_0.pickle"), "rb" ))


FileNotFoundError: [Errno 2] No such file or directory: '/data/chodera/zhangi/perses_benchmark/neq/11/0/0_vacuum_0.pickle'

In [3]:
# Determine indices for solute region
for atom in htf.hybrid_topology.atoms:
    print(atom.index, atom)

0 ACE1-C
1 ACE1-O
2 ACE1-CH3
3 ACE1-H1
4 ACE1-H2
5 ACE1-H3
6 THR2-N
7 THR2-CA
8 THR2-C
9 THR2-O
10 THR2-CB
11 THR2-CG2
12 THR2-OG1
13 THR2-H
14 THR2-HA
15 THR2-HB
16 THR2-HG1
17 THR2-HG21
18 THR2-HG22
19 THR2-HG23
26 THR2-CB
27 THR2-HB1
28 THR2-HB2
29 THR2-HB3
20 NME3-N
21 NME3-C
22 NME3-H
23 NME3-H1
24 NME3-H2
25 NME3-H3


In [4]:
# Check forces to see if CMMForce needs to be removed
for force in range(htf.hybrid_system.getNumForces()):
    print(htf.hybrid_system.getForce(force))

<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2ab18fcf3de0> >
<simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2ab18fcf3de0> >
<simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2ab1c253bb70> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ab18fcf3de0> >


In [5]:
# Build REST factory
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=list(range(6, 10)) + [0, 1, 20, 22])

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


In [6]:
# Get REST system
REST_system = factory.REST_system

In [7]:
# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

In [8]:
# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

In [9]:
# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

In [14]:
# Set up sampler
length = 5 # 5 ns
move = mcmc.GHMCMove(timestep=4.0*unit.femtoseconds, n_steps=250)
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=length*1000)

# Run t-repex
reporter_file = os.path.join(out_dir, f"0_vacuum_thr_{length}ns.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state,
                  storage=reporter)
simulation.run()

DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/11/0/0_vacuum_thr_5ns_checkpoint.nc
DEBUG:mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2ab1e9883090>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2ab1c4d660e0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2ab1c4d17170>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2ab1c4d17440>
DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  5403B | 5.276KB | 0.005MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.034s
DEBUG:openmmtools.multistate.multistatesampler:Sto

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

DEBUG:openmmtools.utils:Computing energy matrix took    8.075s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2ab1e9883090>>
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 1/5000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2ab1c4d56f80>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.029s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 41472/41472 attempted swaps (100.0%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _pr

KeyboardInterrupt: 